# Telecom Customer Churn Prediction Feature Engineering

This tutorial is based on [this](https://www.kaggle.com/pavanraj159/telecom-customer-churn-prediction/comments#6.-Model-Performances) Kaggle notebook and [this](https://github.com/gojek/feast/tree/master/examples/feast-xgboost-churn-prediction-tutorial) Feast notebook

## 1. Churn Feature Engineering

In [1]:
from hops import pandas_helper as pandas
from hops import featurestore
import pandas as pd
import numpy as np
import os
import itertools
import warnings
warnings.filterwarnings("ignore")
import sklearn # Tested with 0.22.1
from slugify import slugify

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
4,application_1591646412804_0014,pyspark,idle,Link,Link


SparkSession available as 'spark'.


### 1.1 Data

In [2]:
#telcom = pd.read_csv('http://feast-examples.storage.googleapis.com/telco-churn-xgboost/telco_customer_churn.csv')
telcom = pandas.read_csv('Resources/churn/telco_customer_churn.csv')
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
telcom.head(5)

An error was encountered:
path hdfs://rpc.namenode.service.consul:8020/Projects/telecom/Resources/telco_customer_churn.csv not found
Traceback (most recent call last):
  File "/srv/hops/anaconda/anaconda/envs/telecom/lib/python3.6/site-packages/hops/pandas_helper.py", line 21, in read_csv
    hdfs_path = hdfs._expand_path(hdfs_filename)
  File "/srv/hops/anaconda/anaconda/envs/telecom/lib/python3.6/site-packages/hops/hdfs.py", line 161, in _expand_path
    raise IOError("path %s not found" % hdfs_path)
OSError: path hdfs://rpc.namenode.service.consul:8020/Projects/telecom/Resources/telco_customer_churn.csv not found



### 1.2 Data Manipulation

In [3]:
# Replacing spaces with null values in total charges column
telcom['total_charges'] = telcom["total_charges"].replace(" ",np.nan)

# Dropping null values from total charges column which contain .15% missing data 
telcom = telcom[telcom["total_charges"].notnull()]
telcom = telcom.reset_index()[telcom.columns]

# Convert to float type
telcom["total_charges"] = telcom["total_charges"].astype(float)

# Replace 'No internet service' to No for the following columns
replace_cols = [ 'online_security', 'online_backup', 'device_protection',
                'tech_support','streaming_tv', 'streaming_movies']
for i in replace_cols : 
    telcom[i]  = telcom[i].replace({'internet_service' : 'No'})
    
# Replace binary values with strings
telcom["senior_citizen"] = telcom["senior_citizen"].replace({1:"Yes",0:"No"})

# Tenure to categorical column
def tenure_lab(telcom) :
    
    if telcom["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (telcom["tenure"] > 12) & (telcom["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (telcom["tenure"] > 24) & (telcom["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (telcom["tenure"] > 48) & (telcom["tenure"] <= 60) :
        return "Tenure_48-60"
    elif telcom["tenure"] > 60 :
        return "Tenure_gt_60"
telcom["tenure_group"] = telcom.apply(lambda telcom:tenure_lab(telcom),
                                      axis = 1)

# Separating churn and non churn customers
churn     = telcom[telcom["churn"] == "Yes"]
not_churn = telcom[telcom["churn"] == "No"]

# Separating catagorical and numerical columns
Id_col     = ['customer_id']
target_col = ["churn"]
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

telcom.head()

An error was encountered:
name 'telcom' is not defined
Traceback (most recent call last):
NameError: name 'telcom' is not defined



### 1.3 Data Processing

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Customer id col
Id_col     = ['customer_id']

# Target columns
target_col = ["churn"]

# Categorical columns
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]

# Numerical columns
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

# Binary columns with 2 values
bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()

# Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

# Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    telcom[i] = le.fit_transform(telcom[i])
    
# Duplicating columns for multi value columns
telcom = pd.get_dummies(data = telcom,columns = multi_cols )

# Scaling Numerical columns
std = StandardScaler()
scaled = std.fit_transform(telcom[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

# Dropping original values merging scaled values for numerical columns
df_telcom_og = telcom.copy()
telcom = telcom.drop(columns = num_cols,axis = 1)
telcom = telcom.merge(scaled,left_index=True,right_index=True,how = "left")

# Clean up column names
telcom.columns = [slugify(col, lowercase=True, separator='_') for col in telcom.columns]
telcom.columns = map(str.lower, telcom.columns)
telcom.head()

An error was encountered:
name 'telcom' is not defined
Traceback (most recent call last):
NameError: name 'telcom' is not defined



### 1.4 Create FeatureGroups

In [5]:
from hops import featurestore
featurestore.create_featuregroup(telcom, "telcom_featuregroup", description="Telecom churn prediction feature group")

An error was encountered:
name 'telcom' is not defined
Traceback (most recent call last):
NameError: name 'telcom' is not defined

